In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import skew
import xgboost as xgb

from sklearn.cross_validation import KFold
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import LogisticRegression

from math import sqrt
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold

from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
import gc



In [ ]:
NFOLDS = 3
SEED = 0
NROWS = None

data = pd.read_csv('../input/application_train.csv')
test = pd.read_csv('../input/application_test.csv')
prev = pd.read_csv('../input/previous_application.csv')


In [ ]:
print('%-55s | %7s | %10s | %10s | %10s' % ('features','type','nb_values','nb_NaNs','NaNs (%)'))

for f_ in data :     #. dtypes
  print('%-55s | %7s | %10s | %10s | %5.2f' % (f_, str(data[f_].dtype), str(len(data[f_].value_counts(dropna=False))),
                                                                                100*data[f_].isnull().sum() / data.shape[0]))



In [ ]:
categorical_feats = [f for f in data.columns if data[f].dtype == 'object']

for f_ in categorical_feats :
  data[f_], indexer = pd.factorize(data[f_])
  test[f_] = indexer.get_indexer(test[f_])

gc.enable()

y_train = data['target']
def data['target']

#averaging factorized categorical features defeats my own reasoning
#DO NOT RUN IT

prev_cat_features = [f for f_ in prev.columns if prev[f_].dtype =='object']

for f_ in prev_cat_features :
  prev[f_], _ = pd.factorize(prev[f_])

avg_prev = prev.groupby('sk_id_curr').mean()
cnt_prev = prev[['sk_id_curr','sk_id_prev']].groupby('sk_id_curr').count()
avg_prev['nb_app'] = cnt_prev['sk_id_prev']
del avg_prev['sk_id_prev']



In [ ]:
x_train = data.merge(right=avg_prev.reset_index(), how='left', on='sk_id_curr')
x_test = test.merge(right=avg_prev.reset_index(), how='left', on='sk_id_curr')

x_train=x_train.fillna(0)
x_test = x_test.fillna(0)

In [ ]:
ntrain = x_train.shape[0]
ntest = x_test.shape[0]

In [ ]:
excluded_feats = ['sk_id_curr']
features = [f_ for f_ in x_train.columns if f_ not in excluded_feats]

x_train=x_train[features]
x_test = x_test[features]

In [1]:
kf=KFold(n_splits=NFOLDS, shuffle=True, random_state=SEED)

class SklearnWrapper(object) :
  def __init__(self, clf, seeld=0, params=None) :
    params['random_state'] = seed
    self.clf=clf(**params)

  def train(self, x_train, y_train) :
    self.clf.fit(x_train, y_train)

  def predict(self, x) :
    return self.clf.predict_proba(x)[:,1]



SyntaxError: ignored

In [ ]:
class CatboostWrapper(object) :
  def __init__(self,clf,seed=0, params=None) :
    params['random_state'] = seed
    self.clf = clf(**params)

  def train(self, x_train, y_train) :
    self.clf.fit(x_train, y_train)

  def predict(self,x) :
    self.clf.predict_proba(x)[:,1]


In [ ]:
class XgbWrapper(object) :
  def __init_(self,seed=0, params=None) :
    self.param = params
    self.param['seed'] = seed
    self.nrounds = params.pop('nrounds', 250)

  def train(self, x_train, y_train) :
    dtrain = xgb.DMatrix(x_train, label = y_train)
    self.gbdt = xgb.train(self.param, dtrain, self.nrounds)

  def predict(self, x) :
    return self.gbdt.predict(xgb.DMatrix(x))



In [ ]:
def get_oof(clf) :
  oof_train = np.zeros((ntrain,))
  oof_test = np.zeros((ntest,))
  oof_test_skf = np.empty((NFOLDS, ntest))

  for i, (train_index, test_index) in enumerate(kf.split(x_train)) :
    x_tr = x_train.loc[train_index]
    y_tr = y_train.loc[train_index]
    x_te = x_train.loc[test_index]

    clf.train(x_tr,y_tr)

    oof_train[test_index] = clf.predict(x_te)
    oof_test_skf[i,:] = clf.predict(x_test)

  oof_test[:] = oof_test.skf.mean(axis=0)
  return oof_train.reshape(-1,1), oof_test.reshape(-1,1)





In [ ]:
et_params = {
    'n_jobs' : 16,
    'n_estimators' : 200,
    'max_features' : 0.5,
    'max_depth' : 12,
    'min_samples_leaf' : 2
}

rf_params = {
    'n_jobs' : 16,
    'n_estimators' : 200,
    'max_features' : 0.5,
    'max_depth' : 12,
    'min_samples_leaf' : 2
}

xgb_params = {
    'seed' : 0,
    'colsample_bytree' : 0.7,
    'silent' : 1,
    'subsample' : 0.7,
    'learning_rate' : 0.075,
    'object' : 'binary:logistic',
    'max_depth' : 4,
    'num_parallel_tree' : 1,
    'min_child_weight' : 1,
    'nrounds' : 200
}

catboost_params = {
    'n_jobs' : 16,
    'n_estimators' : 200,
    'max_features' : 0.5,
    'max_depth' : 12,
    'min_samples_leaf' : 2
}

lightgbm_params = {
    'n_jobs' : 16,
    'n_estimators' : 200,
    'max_features' : 0.5,
    'max_depth' : 12,
    'min_samples_leaf' : 2
}

In [ ]:
xg = XgbWrapper(seed=SEED, params=xg_params)
et = SklearnWrapper(ExtraTreesClassifier, seed=SEED, params=et_params)
rf = SklearnWrapper(RandomForestClassifier, seed=SEED, params=rf_params)
cb=
lg=


xg_oof_train, xg_oof_test = get_oof(xg)
xg_oof_train, xg_oof_test = get_oof(et)
xg_oof_train, xg_oof_test = get_oof(rf)
xg_oof_train, xg_oof_test = get_oof(cb)

In [ ]:
print('XG_CV : {}'.foramt(sqrt(mean_squared_error(y_train, xg_oof_train))))
print('XG_CV : {}'.foramt(sqrt(mean_squared_error(y_train, xg_oof_train))))
print('XG_CV : {}'.foramt(sqrt(mean_squared_error(y_train, xg_oof_train))))
print('XG_CV : {}'.foramt(sqrt(mean_squared_error(y_train, xg_oof_train))))

In [ ]:
x_train = np.concatenate( ( xg_oof_train, et_oof_train, rf_oof_train, cb_oof_train), axis=1)
x_test  = np.concatenate( ( xg_oof_train, et_oof_train, rf_oof_train, cb_oof_train), axis=1)

In [ ]:
print('{}, {}'.foramt(x_train.shape, x_test.shape))

logistic_regression = LogisticRegression()
logistic_regression.fit(x_train,y_train)

test['target'] = logistic_regression.predict_proba(x_test)[:,1]

test[['sk_id_curr', 'target']]